# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV and Perform Basic Data Cleaning

In [3]:
columns = [
    "loan_amnt", "int_rate", "installment", #"home_ownership",
    #"annual_inc", "verification_status", "issue_d", 
    "loan_status",
    #"pymnt_plan", 
    "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    #"initial_list_status", 
    "out_prncp", #"out_prncp_inv",
    "total_pymnt", #"total_pymnt_inv", 
    "total_rec_prncp", #"total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", #"next_pymnt_d",
    "collections_12_mths_ex_med", #"policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit"#, "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [4]:
# Load the data
file_path = Path('./LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
# removed fully paid, or charged off also
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]
fp_mask= df['loan_status'] != 'Fully Paid'
df = df.loc[fp_mask]
co_mask = df['loan_status'] != 'Charged Off'
df = df.loc[co_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100

# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,10500.0,0.1719,375.35,low_risk,27.24,0.0,0.0,8.0,0.0,1609.0,...,0.0,3.0,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0
1,25000.0,0.2000,929.09,low_risk,20.23,0.0,0.0,17.0,1.0,18368.0,...,0.0,6.0,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0
2,20000.0,0.2000,529.88,low_risk,24.26,0.0,0.0,8.0,0.0,13247.0,...,0.0,4.0,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0
3,10000.0,0.1640,353.55,low_risk,31.44,0.0,1.0,10.0,1.0,17996.0,...,0.0,1.0,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0
4,22000.0,0.1474,520.39,low_risk,18.76,0.0,1.0,14.0,0.0,9091.0,...,0.0,1.0,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0


# Split the Data into Training and Testing

In [6]:
# Create our features
X = df.copy()
X = X.drop("loan_status", axis=1)
X.head() 

# Create our target
# using label encoder technique
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df2 = df.copy()

df2['loan_status'] = le.fit_transform(df2['loan_status'])
from pandas import DataFrame
y = DataFrame(df2["loan_status"])

In [23]:
# scale all the X data 
from sklearn.preprocessing import MinMaxScaler

data_scaler = MinMaxScaler(feature_range=(-10, 10))
X_scaled = data_scaler.fit_transform(X)

#converting back to dataframe
X = pd.DataFrame(X_scaled, index=X.index, columns=X.columns)
#rounding to 2 decimal places 
X = X.round(2)
X.head()

,loan_amnt,int_rate,installment,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,total_acc,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,-5.13,-0.99,-5.81,-9.45,-10.0,-10.0,-8.29,-10.0,-9.95,-8.63,...,-10.0,-6.84,6.43,10.0,-10.0,-10.0,-9.62,-9.41,-9.93,-9.13
1,2.31,1.27,0.92,-9.59,-10.0,-10.0,-5.71,-5.0,-9.37,-6.15,...,-10.0,-3.68,7.80,0.0,-5.0,-10.0,-8.37,-9.07,-8.39,-9.31
2,-0.26,1.27,-3.93,-9.51,-10.0,-10.0,-8.29,-10.0,-9.55,-7.76,...,-10.0,-5.79,1.68,0.0,-10.0,-10.0,-9.65,-9.30,-9.71,-9.40
3,-5.38,-1.63,-6.08,-9.37,-10.0,-6.0,-7.71,-5.0,-9.39,-7.52,...,-10.0,-8.95,10.00,0.0,-5.0,-10.0,-9.42,-8.94,-9.23,-8.93
4,0.77,-2.96,-4.05,-9.62,-10.0,-6.0,-6.57,-10.0,-9.69,-6.89,...,-10.0,-8.95,10.00,-10.0,-10.0,-10.0,-8.69,-9.60,-8.92,-9.72


In [8]:
X.describe()

,loan_amnt,int_rate,installment,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,total_acc,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
count,68817.000000,68817.00000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000
mean,-1.959989,-4.54778,-4.532837,-9.563994,-9.758233,-8.009213,-6.975087,-9.369850,-9.400402,-7.308626,...,-9.942113,-7.663124,8.764626,-3.874757,-9.370141,-10.0,-8.744753,-9.056484,-8.836311,-9.220679
std,5.270321,3.87541,3.501522,0.404396,0.797837,3.032486,1.720878,1.683984,0.743748,1.530366,...,0.433903,1.997693,2.081479,6.726293,1.683661,0.0,1.172376,0.886151,1.052289,0.714275
min,-10.000000,-10.00000,-10.000000,-10.000000,-10.000000,-10.000000,-10.000000,-10.000000,-10.000000,-10.000000,...,-10.000000,-10.000000,-10.000000,-10.000000,-10.000000,-10.0,-10.000000,-10.000000,-10.000000,-10.000000
25%,-5.900000,-7.74000,-7.150000,-9.720000,-10.000000,-10.000000,-8.290000,-10.000000,-9.790000,-8.390000,...,-10.000000,-8.950000,8.250000,-10.000000,-10.000000,-10.0,-9.610000,-9.590000,-9.550000,-9.680000
50%,-2.820000,-5.33000,-5.460000,-9.600000,-10.000000,-10.000000,-7.430000,-10.000000,-9.590000,-7.520000,...,-10.000000,-7.890000,10.000000,-6.000000,-10.000000,-10.0,-9.130000,-9.300000,-9.140000,-9.410000
75%,1.790000,-2.29000,-2.500000,-9.470000,-10.000000,-6.000000,-6.000000,-10.000000,-9.260000,-6.520000,...,-10.000000,-6.840000,10.000000,0.000000,-10.000000,-10.0,-8.180000,-8.820000,-8.460000,-8.990000
max,10.000000,10.00000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,...,10.000000,10.000000,10.000000,10.000000,10.000000,-10.0,10.000000,10.000000,10.000000,10.000000


In [9]:
# Check the balance of our target values
# 1 = low risk, 0 = high risk
y['loan_status'].value_counts()

1    68470
0      347
Name: loan_status, dtype: int64

In [10]:
# converting y back to a series
y = y['loan_status']
type(y)

pandas.core.series.Series

In [11]:
# Create X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [12]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

Counter(y_resampled)

Counter({1: 51366, 0: 51366})

In [13]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [14]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.642533678648501

In [15]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[   60,    41],
       [ 5285, 11819]], dtype=int64)

In [16]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print('Naive Random Oversampling')
print(classification_report_imbalanced(y_test, y_pred))

Naive Random Oversampling
                   pre       rec       spe        f1       geo       iba       sup

          0       0.01      0.59      0.69      0.02      0.64      0.41       101
          1       1.00      0.69      0.59      0.82      0.64      0.41     17104

avg / total       0.99      0.69      0.59      0.81      0.64      0.41     17205



### SMOTE Oversampling

In [17]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy=1.0).fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({1: 51366, 0: 51366})

In [18]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [19]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.6328957849012216

In [20]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   58,    43],
       [ 5276, 11828]], dtype=int64)

In [21]:
# Print the imbalanced classification report
print('SMOTE Oversampling')
print(classification_report_imbalanced(y_test, y_pred))

SMOTE Oversampling
                   pre       rec       spe        f1       geo       iba       sup

          0       0.01      0.57      0.69      0.02      0.63      0.39       101
          1       1.00      0.69      0.57      0.82      0.63      0.40     17104

avg / total       0.99      0.69      0.57      0.81      0.63      0.40     17205



# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [24]:
# Resample the data using the ClusterCentroids resampler
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({0: 246, 1: 246})

In [25]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [26]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.6328957849012216

In [27]:
# Display the confusion matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[   64,    37],
       [ 6240, 10864]], dtype=int64)

In [28]:
# Print the imbalanced classification report
print('ClusterCentroids UnderSampling')
print(classification_report_imbalanced(y_test, y_pred))

ClusterCentroids UnderSampling
                   pre       rec       spe        f1       geo       iba       sup

          0       0.01      0.63      0.64      0.02      0.63      0.40       101
          1       1.00      0.64      0.63      0.78      0.63      0.40     17104

avg / total       0.99      0.64      0.63      0.77      0.63      0.40     17205



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [29]:
# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=1)
X_resampled, y_resampled = smote_enn.fit_resample(X, y)
Counter(y_resampled)

Counter({0: 68470, 1: 62148})

In [30]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [31]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.6344182126351082

In [32]:
# Display the confusion matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[   73,    28],
       [ 6305, 10799]], dtype=int64)

In [33]:
# Print the imbalanced classification report
print('Combination SMOTEENN Sampling')
print(classification_report_imbalanced(y_test, y_pred))

Combination SMOTEENN Sampling
                   pre       rec       spe        f1       geo       iba       sup

          0       0.01      0.72      0.63      0.02      0.68      0.46       101
          1       1.00      0.63      0.72      0.77      0.68      0.45     17104

avg / total       0.99      0.63      0.72      0.77      0.68      0.45     17205

